In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from wordcloud import WordCloud
from os import walk
from string import punctuation
from random import shuffle
from collections import Counter
import multiprocessing
import email


import sklearn as sk

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Akhil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Akhil\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Akhil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Akhil\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Akhil\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [2]:
data = pd.read_csv('emails.csv')
data.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [3]:
data.columns

Index(['text', 'spam'], dtype='object')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5728 non-null   object
 1   spam    5728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.6+ KB


In [9]:
data.shape

(5728, 2)

In [11]:
data1 = data.copy()
df = data1

### Data cleaning


In [13]:
# Punctuation removal

import string
# Function to remove punctuations.
def remove_punc(text):
    nonP_text = "".join([char for char in text if char not in string.punctuation])
    return nonP_text

df["text_clean"] = df["text"].apply(lambda x: remove_punc(x))

df.head()

,text,spam,text_clean
0,Subject: naturally irresistible your corporate...,1,Subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,1,Subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,1,Subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,1,Subject 4 color printing special request addi...
4,"Subject: do not have money , get software cds ...",1,Subject do not have money get software cds fr...


In [14]:
# tokenization

import re

#function to apply tokenization
def tokenize(text):
    tokens = re.split("\W+", text)# W+ means all capital, small alphabets and integers 0-9
    return tokens

df["tokens"] = df["text_clean"].apply(lambda x: tokenize(x))

df.head()

,text,spam,text_clean,tokens
0,Subject: naturally irresistible your corporate...,1,Subject naturally irresistible your corporate ...,"[Subject, naturally, irresistible, your, corpo..."
1,Subject: the stock trading gunslinger fanny i...,1,Subject the stock trading gunslinger fanny is...,"[Subject, the, stock, trading, gunslinger, fan..."
2,Subject: unbelievable new homes made easy im ...,1,Subject unbelievable new homes made easy im w...,"[Subject, unbelievable, new, homes, made, easy..."
3,Subject: 4 color printing special request add...,1,Subject 4 color printing special request addi...,"[Subject, 4, color, printing, special, request..."
4,"Subject: do not have money , get software cds ...",1,Subject do not have money get software cds fr...,"[Subject, do, not, have, money, get, software,..."


In [15]:
# removal of stopwords

import nltk
stopwords = nltk.corpus.stopwords.words("english")

def remove_stopwords(token):
    text = [word for word in token if word not in stopwords] # to remove all stopwords
    return text

df["text_nonstop"] = df["tokens"].apply(lambda x: remove_stopwords(x))
df.head()

,text,spam,text_clean,tokens,text_nonstop
0,Subject: naturally irresistible your corporate...,1,Subject naturally irresistible your corporate ...,"[Subject, naturally, irresistible, your, corpo...","[Subject, naturally, irresistible, corporate, ..."
1,Subject: the stock trading gunslinger fanny i...,1,Subject the stock trading gunslinger fanny is...,"[Subject, the, stock, trading, gunslinger, fan...","[Subject, stock, trading, gunslinger, fanny, m..."
2,Subject: unbelievable new homes made easy im ...,1,Subject unbelievable new homes made easy im w...,"[Subject, unbelievable, new, homes, made, easy...","[Subject, unbelievable, new, homes, made, easy..."
3,Subject: 4 color printing special request add...,1,Subject 4 color printing special request addi...,"[Subject, 4, color, printing, special, request...","[Subject, 4, color, printing, special, request..."
4,"Subject: do not have money , get software cds ...",1,Subject do not have money get software cds fr...,"[Subject, do, not, have, money, get, software,...","[Subject, money, get, software, cds, software,..."


In [16]:
ps = nltk.PorterStemmer()

def stemming(t_text):
    text = [ps.stem(word) for word in t_text]
    return text

df["text_stemmed"] = df["text_nonstop"].apply(lambda x: stemming(x))
df.head()

,text,spam,text_clean,tokens,text_nonstop,text_stemmed
0,Subject: naturally irresistible your corporate...,1,Subject naturally irresistible your corporate ...,"[Subject, naturally, irresistible, your, corpo...","[Subject, naturally, irresistible, corporate, ...","[subject, natur, irresist, corpor, ident, lt, ..."
1,Subject: the stock trading gunslinger fanny i...,1,Subject the stock trading gunslinger fanny is...,"[Subject, the, stock, trading, gunslinger, fan...","[Subject, stock, trading, gunslinger, fanny, m...","[subject, stock, trade, gunsling, fanni, merri..."
2,Subject: unbelievable new homes made easy im ...,1,Subject unbelievable new homes made easy im w...,"[Subject, unbelievable, new, homes, made, easy...","[Subject, unbelievable, new, homes, made, easy...","[subject, unbeliev, new, home, made, easi, im,..."
3,Subject: 4 color printing special request add...,1,Subject 4 color printing special request addi...,"[Subject, 4, color, printing, special, request...","[Subject, 4, color, printing, special, request...","[subject, 4, color, print, special, request, a..."
4,"Subject: do not have money , get software cds ...",1,Subject do not have money get software cds fr...,"[Subject, do, not, have, money, get, software,...","[Subject, money, get, software, cds, software,...","[subject, money, get, softwar, cd, softwar, co..."


In [19]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(t_text):
    text = [wn.lemmatize(word) for word in t_text]
    return text

df["text_lemmatized"] = df["text_stemmed"].apply(lambda x: lemmatizer(x))
df.head()

,text,spam,text_clean,tokens,text_nonstop,text_stemmed,text_lemmatized
0,Subject: naturally irresistible your corporate...,1,Subject naturally irresistible your corporate ...,"[Subject, naturally, irresistible, your, corpo...","[Subject, naturally, irresistible, corporate, ...","[subject, natur, irresist, corpor, ident, lt, ...","[subject, natur, irresist, corpor, ident, lt, ..."
1,Subject: the stock trading gunslinger fanny i...,1,Subject the stock trading gunslinger fanny is...,"[Subject, the, stock, trading, gunslinger, fan...","[Subject, stock, trading, gunslinger, fanny, m...","[subject, stock, trade, gunsling, fanni, merri...","[subject, stock, trade, gunsling, fanni, merri..."
2,Subject: unbelievable new homes made easy im ...,1,Subject unbelievable new homes made easy im w...,"[Subject, unbelievable, new, homes, made, easy...","[Subject, unbelievable, new, homes, made, easy...","[subject, unbeliev, new, home, made, easi, im,...","[subject, unbeliev, new, home, made, easi, im,..."
3,Subject: 4 color printing special request add...,1,Subject 4 color printing special request addi...,"[Subject, 4, color, printing, special, request...","[Subject, 4, color, printing, special, request...","[subject, 4, color, print, special, request, a...","[subject, 4, color, print, special, request, a..."
4,"Subject: do not have money , get software cds ...",1,Subject do not have money get software cds fr...,"[Subject, do, not, have, money, get, software,...","[Subject, money, get, software, cds, software,...","[subject, money, get, softwar, cd, softwar, co...","[subject, money, get, softwar, cd, softwar, co..."
